## JuPedSim

In diesem Tutorial wird eine Simulation mit `jpscore` in einem einfachen Setup durchgeführt. 

Eine Dokumentation der verwendeten Dateien ist unter:

- Die Trajektorie-Datei (txt oder xml) (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_trajectory.html))
- Die Geometriedatei (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_geometry.html)) 
- Die Projektdatei von jpscore, genannt inifile (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_inifile.html))


## Wie benutze ich ein Jupyter-Notebook

Dieser `Jupyter-Notebook` besteht aus verschiedennen Zellen (Cells). 

Sie können alle Zellen ausführen indem Sie in dem Menü `Run --> Run All Cells` auswählen.

Alternative, können Sie einzelne Zellen ausführen, indem Sie auf die entsprechende Zelle klicken und `Run --> Run Selected Cells` auswählen. 

**Tip**: Die Ausführung von einer ausgewählten Zelle geht auch mit dem Kürzel `Shift+Enter` (Also auf den Tasten `Alt` und `Enter` gleichzeitig drücken)


## Install 

In [ ]:
%run 00_Install.ipynb

## Simulation

Mit den Dateien `ini_jpscore.xml` und `geometry.xml` können wir eine einfache Simulation wie folgt durchführen:

In [ ]:
%%bash
$HOME/usr/bin/jpscore ini_jpscore.xml

In [ ]:
inifile = "ini_jpscore.xml"
d = parse(inifile)
Npeds = int(dict(d.getElementsByTagName('group')[0].attributes.items())['number'])
print("Npeds:", Npeds)

# Visualisierung der Trajektorien

*Warnung*: Die Visualisierung einer großen Trajektoriendatei kann einige Minuten dauern. Also, Geduld! 

Der Name der Geometrie- und Trajektorie-Dateien werden automatisch aus der Ini-Datei extrahiert. 
Darüber hinaus wird auch der Wert der Frames pro Sekunde (*fps*) aus der Ini-Datei extrahiert.

In [ ]:
%%time
%run 02_vis.ipynb
HTML(ani.to_html5_video())

----

## N(t)-Kurve zeichnen

In dem Ordner `results` wird eine Flow-Datei erzeugt, die zwei Spalten enthält:

Abhängig von der Trajektoriennamen könnte diese Datei so heissen `flow_exit_id_0_trajectories.txt`.

**Achtung:**
Wenn die Trajektoriendatei in der `ini_jpscore.xml` geändert wird, muss der Wert der Variable `Nt_file` (siehe Zelle unten) dementsprechend **angepasst** werden.

Zum Beispiel: 

| Trajektorienname  |  Flow-Datei |
| ------------- |:-------------:|
| trajectories50.txt | flow_exit_id_0_trajectories50.txt|
| trajectories100.txt | flow_exit_id_0_trajectories100.txt|
| trajectories150.txt | flow_exit_id_0_trajectories150.txt|



In [ ]:
%run libs/imports
%matplotlib inline
import glob
files = glob.glob("results/flow*.txt")

for f in files:
    print()
    data = np.loadtxt(f)
    plt.plot(data[:, 1],data[:,0],label=f.split("_")[-1].split("trajectories.txt")[0])
    
plt.ylabel("$t$", size=18)
plt.xlabel("$N$", size=18)
plt.grid()
plt.legend()

## jpsreport


In [ ]:
%%bash
$HOME/usr/bin/jpsreport ini_jpsreport.xml

In [ ]:
Nt_file="Output/Fundamental_Diagram/FlowVelocity/Flow_NT_%strajectories.txt_id_2.dat"%Npeds
if not os.path.exists(Nt_file):
    print("File {} does not exist".format(Nt_file))
else:
    data = np.loadtxt(Nt_file)
    plt.plot(data[:, 1],data[:,0])
    plt.ylabel("$t$", size=18)
    plt.xlabel("$N$", size=18)
    plt.grid()

In [ ]:
files = glob.glob("Output/Fundamental_Diagram/FlowVelocity/Flow_NT_*trajectories.txt_id_2.dat")
for f in files:
    data = np.loadtxt(f)
    plt.plot(data[:, 1],data[:,0], label=f.split("trajectories.txt")[0].split("_")[-1])
    
plt.ylabel("$t$", size=18)
plt.xlabel("$N$", size=18)
plt.legend()
plt.grid()

## Aufräumen

Dieser Schritt ist **optional**. 

Um den Simulationsordners `results` zu löschen, entfernen Sie von der Zelle unten das Zeichen `#` und führen Sie anschliessend die Zelle mit `Alt+Enter` 

In [ ]:
%%bash
#rm -r results 